## 1. Import libraries and dependencies

In [1]:
from src.preprocess.preprocess import Preprocess
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import time

2024-05-23 11:48:16.807835: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-23 11:48:16.810539: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-23 11:48:16.845975: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-23 11:48:17.497095: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


/home/hubert/Workspace/Python/ML Project/languages
{'cs': '/home/hubert/Workspace/Python/ML Project/languages/cs', 'lt': '/home/hubert/Workspace/Python/ML Project/languages/lt', 'pl': '/home/hubert/Workspace/Python/ML Project/languages/pl'}
{'cs': '/home/hubert/Workspace/Python/ML Project/languages/cs/validated.tsv', 'lt': '/home/hubert/Workspace/Python/ML Project/languages/lt/validated.tsv', 'pl': '/home/hubert/Workspace/Python/ML Project/languages/pl/validated.tsv'}
{'cs': '/home/hubert/Workspace/Python/ML Project/languages/cs/clip_durations.tsv', 'lt': '/home/hubert/Workspace/Python/ML Project/languages/lt/clip_durations.tsv', 'pl': '/home/hubert/Workspace/Python/ML Project/languages/pl/clip_durations.tsv'}


2024-05-23 11:48:18.142302: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


## 2. Preprocess audio

In [2]:
ORIGIN_SAMPLE_RATE = 48_000
FINAL_SAMPLE_RATE = 16_000
MAX_CLIENT_ID_AMOUNT = 3000
MIN_CLIP_DURATION_MS = 2000
SET_SIZE = 12_000
BATCH_SIZE = 32

In [3]:
Preprocess.set_origin_sample_rate(ORIGIN_SAMPLE_RATE)
Preprocess.set_final_sample_rate(FINAL_SAMPLE_RATE)
Preprocess.set_max_client_id_amount(MAX_CLIENT_ID_AMOUNT)
Preprocess.set_min_clip_duration_ms(MIN_CLIP_DURATION_MS)
Preprocess.set_set_size(SET_SIZE)
Preprocess.set_batch_size(BATCH_SIZE)

<class 'int'>


In [4]:
print(f"Original sample rate: {Preprocess.ORIGIN_SAMPLE_RATE}")
print(f"Final sample rate: {Preprocess.SAMPLE_RATE}")
print(f"Max client id amount: {Preprocess.MAX_CLIENT_ID_AMOUNT}")
print(f"Min clip duration in milliseconds: {Preprocess.MIN_CLIP_DURATION_MS}")
print(f"Set size: {Preprocess.SET_SIZE}")
print(f"One language set size: {Preprocess.ONE_LANG_SET_SIZE}")

print("-"*25 + "SETS SIZES" + "-"*25)
print(f"Training set size: {Preprocess.SET_HPARAMS.train_size}")
print(f"Val set size: {Preprocess.SET_HPARAMS.val_size}")
print(f"Test set size: {Preprocess.SET_HPARAMS.test_size}")
print(f"Is test + val + train is equal one lang set size: "
      f"\n{Preprocess.SET_HPARAMS.test_size + Preprocess.SET_HPARAMS.val_size + Preprocess.SET_HPARAMS.train_size} <= {Preprocess.ONE_LANG_SET_SIZE}")

Original sample rate: 48000
Final sample rate: 16000
Max client id amount: 3000
Min clip duration in milliseconds: 2000
Set size: 12000
One language set size: 4000
-------------------------SETS SIZES-------------------------
Training set size: 2400
Val set size: 800
Test set size: 800
Is test + val + train is equal one lang set size: 
4000 <= 4000


In [5]:
start = time.time()
Preprocess.split_filenames()
end = time.time()
print(f"Preprocess time: {end - start}")
print(len(Preprocess.TRAIN_FILENAMES))
print(len(Preprocess.VAL_FILENAMES))
print(len(Preprocess.TEST_FILENAMES))

PROCESSED LANGUAGES: cs
---AUDIO META INFO DONE cs---
1200
400
400
---SET SPLIT DONE 0 cs---
Missing probes: 0
Tuned train size: 1200
Tuned val size: 400
Tuned test size: 400
--------------------------
Test size:  400
Val size:  400
Train size:  1200
--------------------------
--------------------------
1200
400
400
---SET SPLIT DONE 1 cs---
Missing probes: 0
Tuned train size: 1200
Tuned val size: 400
Tuned test size: 400
--------------------------
Test size:  400
Val size:  400
Train size:  1200
--------------------------
--------------------------
PROCESSED LANGUAGES: lt
---AUDIO META INFO DONE lt---
1200
400
400
---SET SPLIT DONE 0 lt---
Missing probes: 0
Tuned train size: 1200
Tuned val size: 400
Tuned test size: 400
--------------------------
Test size:  400
Val size:  400
Train size:  1200
--------------------------
--------------------------
1200
400
400
---SET SPLIT DONE 1 lt---
Missing probes: 0
Tuned train size: 1200
Tuned val size: 400
Tuned test size: 400
------------------

In [6]:
def _set_train_val_test_sizes(_set_size) -> None:
    _train_size = int(_set_size * 0.6) if _set_size % 2 == 0 else int(_set_size * 0.6) - 1
    _val_size = int((_set_size - _train_size) // 2) if _train_size % 2 == 0 else int((_set_size - _train_size) // 2) - 1
    _test_size = _val_size
    print(_train_size)
    print(_val_size)
    print(_test_size)
    
_set_train_val_test_sizes(650 // 3)

129
42
42


In [7]:
# for audio, label in Preprocess.train_dataset.take(120):
#     print(audio.shape)

In [8]:
# for audio, label in Preprocess.val_dataset.take(112):
#     print(audio.shape)

In [9]:
# for audio, label in Preprocess.test_dataset.take(112):
#     print(audio.shape)

In [10]:
counter = 0
start = time.time()
for batch, label in Preprocess.train_dataset:
    print(batch.shape)
    for sample in batch:
        print(sample)
        counter += 1
end = time.time()
print(f"Train it time: {end - start}")
print(counter)


Train it time: 0.004559516906738281
0


2024-05-23 11:48:24.182545: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [11]:
counter = 0
start = time.time()
for batch, label in Preprocess.val_dataset:
    print(batch.shape)
    for sample in batch:
        print(sample)
        counter += 1
end = time.time()
print(f"Val it time: {end - start}")
print(counter)

Val it time: 0.002330780029296875
0


2024-05-23 11:48:24.229142: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [12]:
counter = 0
start = time.time()
for batch, label in Preprocess.test_dataset:
    print(batch.shape)
    for sample in batch:
        print(sample)
        counter += 1
end = time.time()
print(f"Test it time: {end - start}")
print(counter)

Test it time: 0.0024290084838867188
0


2024-05-23 11:48:24.277822: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [13]:
Preprocess.train_dataset.as_numpy_iterator().next()

2024-05-23 11:48:24.324352: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


StopIteration: 

In [ ]:
samples, labels = Preprocess.train_dataset.as_numpy_iterator().next()
print()
print(samples.shape, labels.shape)
samples, labels = Preprocess.val_dataset.as_numpy_iterator().next()
print(samples.shape, labels.shape)
samples, labels = Preprocess.test_dataset.as_numpy_iterator().next()
print(samples.shape, labels.shape)

In [ ]:
samples, labels = Preprocess.train_dataset.as_numpy_iterator().next()

In [ ]:
samples.shape

In [ ]:
input_shape = samples.shape[1:]
input_shape

In [ ]:
Preprocess.test_dataset.element_spec

In [ ]:
Preprocess.val_dataset.element_spec

In [ ]:
Preprocess.train_dataset.element_spec

## 4. Create model

In [ ]:
# model = keras.Sequential([
#     layers.Input(shape=input_shape),
#     layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
#     layers.MaxPooling2D((2, 2)),
#     layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
#     layers.MaxPooling2D((2, 2)),
#     layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
#     layers.Flatten(),
#     layers.Dense(64, activation='relu'),
#     layers.Dense(3, activation='softmax')
# ])
# model.summary()

In [ ]:
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
Preprocess.train_dataset.element_spec

In [ ]:
Preprocess.val_dataset.element_spec

In [ ]:
# history = model.fit(Preprocess.train_dataset, epochs=10, validation_data=Preprocess.val_dataset, batch_size=BATCH_SIZE)

In [ ]:
# test_loss, test_acc = model.evaluate(Preprocess.test_dataset, verbose=2)
# print(f'\nTest accuracy: {test_acc}')

In [ ]:
# plt.plot(history.history['accuracy'], label='accuracy')
# plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.ylim([0.5, 1])
# plt.legend(loc='lower right')
# plt.show()
